In [12]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

In [13]:
def fetch_data():
    file_path = "companies_esg_scores.xlsx"  
    df = pd.read_excel(file_path, header=0)
    return df

In [14]:
df = fetch_data()
e_score = df["E_Score"].iloc[-1]
s_score = df["S_Score"].iloc[-1]
g_score = df["G_Score"].iloc[-1]

In [15]:
e_metrics = ["Emission_intensities", "Energy_consumption_intensity", "Waste_generated", "Water_intensity"]
s_metrics = ["Average_training_hours_per_employee", "Current_employees_by_age_groups", "Current_employees_by_gender",
             "Employee_satisfaction_rate", "New_hires_by_gender", "New_hires_by_age",
             "Total_turnover", "Turnover_by_gender", "Turnover_by_age", "Fatalities",
             "Percentage_of_employees_covered_by_health_insurance", "High_consequence_injuries",
             "Work_related_injuries", "Company_donated"]
g_metrics = ["Board_independence", "Women_in_the_management_team", "Women_on_the_board"]

In [16]:
# Horizontal bar chart of e_score, s_score and g_score
def create_bar_chart(e_score, s_score, g_score):
    fig_bar = go.Figure()
    fig_bar.add_trace(go.Bar(
        x = [e_score, s_score, g_score],
        y = ["Environment", "Social", "Governance"],
        marker = dict(color = "lightgreen"),
        orientation = "h"
))
    return fig_bar

In [17]:
# Polar chart of indicators
def create_polar_chart(values, metrics, title):
    sorted_data = sorted(zip(values, metrics), reverse=True)
    sorted_values, sorted_metrics = zip(*sorted_data)
    fig = go.Figure(go.Barpolar(
        r = values,
        theta = metrics,
        marker = dict(
            color = values,
            colorscale = "Greens",  
            cmin = 0,
            cmax = max(values),
            line = dict(color = "darkgreen", width = 1)  
        ),
        text = metrics,
        hoverinfo = "text+r"
    ))

    fig.update_layout(
        title = title,
        polar = dict(
            radialaxis = dict(visible = True, range = [0, max(values)], showticklabels = False),  
            angularaxis = dict(showline = False, tickfont = dict(size=12), showticklabels = False)  
        ),
        showlegend = False
    )
    return fig

In [18]:
# Create a Dash app
app = Dash(__name__)

In [19]:
app.layout = html.Div([
    dcc.Graph(id = "polar-e-chart"),  
    dcc.Graph(id = "polar-s-chart"),  
    dcc.Graph(id = "polar-g-chart"),  
    dcc.Graph(id = "bar-chart"),  
    dcc.Input(id = "dummy-input", type = "hidden", value = "dummy")  
])

In [20]:
@app.callback(
    [Output("polar-e-chart", "figure"),
     Output("polar-s-chart", "figure"),
     Output("polar-g-chart", "figure"),
     Output("bar-chart", "figure")],
     Input("dummy-input", "value")
)

def update_charts(selected_metric):
    df = fetch_data()  
    e_score = df["E_Score"].iloc[-1]
    s_score = df["S_Score"].iloc[-1]
    g_score = df["G_Score"].iloc[-1]
    
    bar_fig = create_bar_chart(e_score, s_score, g_score)
    
     # Polar chart of environment indicators
    e_values = [df[m].iloc[-1] for m in e_metrics]
    polar_e_fig = create_polar_chart(e_values, e_metrics, "Environment Metrics")

    # Polar chart of social indicators
    s_values = [df[m].iloc[-1] for m in s_metrics]
    polar_s_fig = create_polar_chart(s_values, s_metrics, "Social Metrics")

    # Polar chart of governance indicators
    g_values = [df[m].iloc[-1] for m in g_metrics]
    polar_g_fig = create_polar_chart(g_values, g_metrics, "Governance Metrics")
    
    return polar_e_fig, polar_s_fig, polar_g_fig, bar_fig

In [21]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 8059)